## Comparing scheduling results

This document examines the schedule-length of SDP offline computing by comparing the SDP parametric model approach to a workflow scheduling approach. 

The method used by the parametric model follows a simple division between total compute requirements and total compute provided. Workflow scheduling evalutes the allocation of tasks to machines, which takes into account both potential parallelism in the task workflows (which may reduce overall runtime), as well as bottlenecks in the task workflow (which can increase runtime). Further experiments will also dive into the effects of taking into account network bandwidth and inter-task communication time costs, which is not made explicit in the parametric analysis. 

In [1]:
import sdp_par_model
import random
from sdp_par_model.parameters.definitions import HPSOs
import pandas as pd
import skaworkflows.workflow.hpso_to_observation as hto
import skaworkflows.workflow.workflow_analysis as wa
import skaworkflows.parametric_runner as pr 

from pathlib import Path

from skaworkflows.config_generator import create_config, config_to_shadow
from skaworkflows.hpconfig.specs.sdp import SDP_PAR_MODEL
from shadow.models.workflow import Workflow as ShadowWorkflow
from shadow.models.environment import Environment as ShadowEnvironment
from shadow.algorithms.heuristic import heft as ShadowHEFT

Path.cwd()

PosixPath('/home/rwb/github/thesis_experiments/notebooks/sdp_comparison')

In [3]:
# Paths are relative to cwd
HPSO_PATH = Path('single_hpso.json')
OUTPUT_PATH = Path('output/')

We also need access to some the data that is being used that was generated by the parametric model:

In [4]:
MAX_TEL_DEMAND = 512 # stations 
# Sizing of each component e.g. Gridding, Subtract etc.
COMPONENT_SIZING = Path('../../../skaworkflows/skaworkflows/data/pandas_sizing/component_compute_SKA1_Low.csv')
# Total requirements e.g. HPSO01 DPrepA total FLOPS/sec observatoin
TOTAL_SIZING = Path('../../../skaworkflows/skaworkflows/data/pandas_sizing/total_compute_SKA1_Low.csv')
f"Paths exist: {COMPONENT_SIZING.exists() and TOTAL_SIZING.exists()}"

'Paths exist: True'

### Cluster 
This is the compute infrastructure that is we run the telescope on. The purpose of this test is to compare SDP processing times, between a global compute approach and a node-based compute approach (used in workflow scheduling). We use values derived from the cost-estimates and the SDP parametric model. 

In [5]:
CLUSTER = SDP_PAR_MODEL()
# Values used in itemised, node-based compute
CLUSTER.to_df()

,Telescope,Number of nodes,GPU/Node,Memory/Node (GB),Storage/Node (TB),FLOPS/Node,Total Storage (PB),Total Compute
0,Low,896,2,320,75,31,67,9.610496


In [6]:
# Values used for the global compute
CLUSTER.generate_parametric_model_values()

,PFLOP/s,Input (Cold) Buffer (PB),Hot Buffer (PB),Delivery Buffer (PB),Hot Rate (GB/s per 10TB NVMe SSD),"Hot rate (TB/s, global)",Cold rate (GB/s) per 16TB SATA SSD),"Cold Rate (TB/s, Global)"
0,9.623,43.35,25.5,0.656,3,7.65,0.5,1.35


In [7]:
CLUSTER.buffer_ratio, CLUSTER.memory_bandwidth

((37.94, 100), 8537946428)

In [8]:
CLUSTER.total_compute

9610496000000000.0

In [11]:
import logging
logging.basicConfig(level=logging.INFO)

OUTPUT = 'output'
p = create_config( MAX_TEL_DEMAND, HPSO_PATH, OUTPUT_PATH, COMPONENT_SIZING, TOTAL_SIZING, CLUSTER, 
    timestep='seconds',
    data=False # No data on edges
)
p, list(Path(p.parent / "workflows").iterdir())

INFO:skaworkflows.config_generator:	Telescope: SKA_LOW 
	Creating config with:
	Output Directory: output
	Buffer ratio: (37.94, 100)
	Timestep: seconds
	Data: False
INFO:skaworkflows.config_generator:Reading system sizing...
INFO:skaworkflows.config_generator:Producing the instrument config
INFO:skaworkflows.config_generator:Producing buffer config
INFO:skaworkflows.config_generator:Putting it all together...
INFO:skaworkflows.config_generator:Writing final config to output/config.json
INFO:skaworkflows.config_generator:Configuration generation complete!


(PosixPath('output/config.json'),
 [PosixPath('output/workflows/hpso02b_time-18000_channels-256_tel-512_no_data.json'),
  PosixPath('output/workflows/hpso01_time-18000_channels-256_tel-512_no_data.json'),
  PosixPath('output/workflows/hpso02a_time-18000_channels-256_tel-512_no_data.json')])

In [9]:
# Translate config so it works for shadow library
sp = config_to_shadow(p, 'shadow_')
sp

PosixPath('output/shadow_config.json')

In [10]:
# Total FLOPS for the workflow graph
wa.calculate_total_flops('output/workflows/hpso01_time-18000_channels-256_tel-512_no_data.json')

3.08834201893933e+20

In [11]:
# Total FLOPS based on System Sizing
# Open Parametric model results
wfs =  ['ICAL [Pflop/s]', 'DPrepA [Pflop/s]','DPrepB [Pflop/s]', 'DPrepC [Pflop/s]', 'DPrepD [Pflop/s]']
hpso = 'hpso01'
duration = 18000
wa.calculate_expected_flops(hpso, wfs, duration, TOTAL_SIZING)

0    3.088342e+20
dtype: float64

In [12]:
random.seed(0)
LONG_SYSTEM_SIZING = Path(
        "/home/rwb/github/sdp-par-model/2021-06-02_LongBaseline_HPSOs.csv"
)
hpsos = [HPSOs.hpso02a,  HPSOs.hpso01, HPSOs.hpso02b]
scenario = 'low-adjusted'
result = pr.calculate_parametric_runtime_estimates(
    LONG_SYSTEM_SIZING, scenario, hpsos
)
for hpso in result:
    print(f"{hpso} {result[hpso]['total_flops']:e}")

1761165575877839
7861834424122161 9623000000000000 1761165575877839
ICAL + DPrepA + DPrepB + DPrepC + DPrepD: 2.572392e+20 32720
1761165575877839
7861834424122161 9623000000000000 1761165575877839
ICAL + DPrepA + DPrepB + DPrepC + DPrepD: 3.088050e+20 39279
1761165575877839
7861834424122161 9623000000000000 1761165575877839
ICAL + DPrepA + DPrepB + DPrepC + DPrepD: 2.572392e+20 32720
hpso02a 2.572392e+20
hpso01 3.088050e+20
hpso02b 2.572392e+20


In [13]:
from multiprocessing import Pool
from copy import deepcopy
from itertools import product
from shadow.heft_multiprocessing import run_workflow

# def run_workflow(workflow, environment):
#     workflow.add_environment(environment)
#     print(ShadowHEFT(HEFTWorkflow))

env = ShadowEnvironment("output/shadow_config.json")
wf_path = Path(p.parent / "workflows")
# files = list(wf_path.iterdir())
wfs = [ShadowWorkflow("/home/rwb/github/thesis_experiments/skaworkflows_tests/workflows/hpso01_time-3600_channels-256_tel-512.json") for i in range(3)]
environs = [deepcopy(env) for i in range(3)]
params = list(zip(wfs, environs))
with Pool(processes=4) as pool:
    pool.starmap(run_workflow, params)

# HEFTWorkflow = ShadowWorkflow("/home/rwb/github/thesis_experiments/skaworkflows_tests/workflows/hpso01_time-3600_channels-256_tel-512.json")
# env = ShadowEnvironment("/home/rwb/github/thesis_experiments/skaworkflows_tests/shadow_config.json")
# HEFTWorkflow.add_environment(env)
# test = ShadowHEFT(HEFTWorkflow)

DEBUG:shadow.models.workflow:Tasks do not have pre calculated runtimes
DEBUG:shadow.algorithms.heuristic:Ranking tasks
Ranking tasks:   0%|                                                                                                                                                                                                                         | 0/4866 [00:00<?, ?Tasks/s]DEBUG:shadow.algorithms.heuristic:Ranked 29/4866
DEBUG:shadow.algorithms.heuristic:Ranked 32/4866
DEBUG:shadow.algorithms.heuristic:Ranked 101/4866
DEBUG:shadow.models.workflow:Tasks do not have pre calculated runtimes
DEBUG:shadow.algorithms.heuristic:Ranked 132/4866
DEBUG:shadow.algorithms.heuristic:Ranked 137/4866
DEBUG:shadow.algorithms.heuristic:Ranking tasks
DEBUG:shadow.algorithms.heuristic:Ranked 142/4866
DEBUG:shadow.algorithms.heuristic:Ranked 152/4866
DEBUG:shadow.algorithms.heuristic:Ranked 159/4866
DEBUG:shadow.algorithms.heuristic:Ranked 160/4866
Ranking tasks:   0%|                             

DEBUG:shadow.algorithms.heuristic:Ranked 724/4866
DEBUG:shadow.algorithms.heuristic:Ranked 736/4866
DEBUG:shadow.algorithms.heuristic:Ranked 741/4866
DEBUG:shadow.algorithms.heuristic:Ranked 1046/4866
DEBUG:shadow.algorithms.heuristic:Ranked 733/4866
DEBUG:shadow.algorithms.heuristic:Ranked 770/4866
DEBUG:shadow.algorithms.heuristic:Ranked 1062/4866
DEBUG:shadow.algorithms.heuristic:Ranked 1069/4866
DEBUG:shadow.algorithms.heuristic:Ranked 1077/4866
DEBUG:shadow.algorithms.heuristic:Ranked 1094/4866
Ranking tasks:   1%|█▋                                                                                                                                                                                                             | 39/4866 [00:00<00:13, 364.38Tasks/s]DEBUG:shadow.algorithms.heuristic:Ranked 736/4866
DEBUG:shadow.algorithms.heuristic:Ranked 1121/4866
DEBUG:shadow.algorithms.heuristic:Ranked 741/4866
DEBUG:shadow.algorithms.heuristic:Ranked 1151/4866
DEBUG:shadow.algorithms.heur

DEBUG:shadow.algorithms.heuristic:Ranked 1489/4866
DEBUG:shadow.algorithms.heuristic:Ranked 1916/4866
DEBUG:shadow.algorithms.heuristic:Ranked 1495/4866
DEBUG:shadow.algorithms.heuristic:Ranked 1507/4866
DEBUG:shadow.algorithms.heuristic:Ranked 1923/4866
DEBUG:shadow.algorithms.heuristic:Ranked 1509/4866
DEBUG:shadow.algorithms.heuristic:Ranked 1495/4866
DEBUG:shadow.algorithms.heuristic:Ranked 1535/4866
DEBUG:shadow.algorithms.heuristic:Ranked 1507/4866
Ranking tasks:   2%|████▊                                                                                                                                                                                                         | 114/4866 [00:00<00:12, 379.94Tasks/s]DEBUG:shadow.algorithms.heuristic:Ranked 1509/4866
DEBUG:shadow.algorithms.heuristic:Ranked 1567/4866
DEBUG:shadow.algorithms.heuristic:Ranked 1929/4866
DEBUG:shadow.algorithms.heuristic:Ranked 1570/4866
DEBUG:shadow.algorithms.heuristic:Ranked 1941/4866
DEBUG:shadow.algorith

DEBUG:shadow.algorithms.heuristic:Ranked 2115/4866
DEBUG:shadow.algorithms.heuristic:Ranked 2399/4866
DEBUG:shadow.algorithms.heuristic:Ranked 2085/4866
DEBUG:shadow.algorithms.heuristic:Ranked 2124/4866
DEBUG:shadow.algorithms.heuristic:Ranked 2115/4866
DEBUG:shadow.algorithms.heuristic:Ranked 2130/4866
DEBUG:shadow.algorithms.heuristic:Ranked 2132/4866
Ranking tasks:   3%|█████▍                                                                                                                                                                                                        | 129/4866 [00:00<00:15, 306.61Tasks/s]DEBUG:shadow.algorithms.heuristic:Ranked 2124/4866
DEBUG:shadow.algorithms.heuristic:Ranked 2411/4866
DEBUG:shadow.algorithms.heuristic:Ranked 2139/4866
DEBUG:shadow.algorithms.heuristic:Ranked 2130/4866
DEBUG:shadow.algorithms.heuristic:Ranked 2147/4866
DEBUG:shadow.algorithms.heuristic:Ranked 2431/4866
DEBUG:shadow.algorithms.heuristic:Ranked 2132/4866
DEBUG:shadow.algorith

DEBUG:shadow.algorithms.heuristic:Ranked 2569/4866
DEBUG:shadow.algorithms.heuristic:Ranked 2555/4866
DEBUG:shadow.algorithms.heuristic:Ranked 2795/4866
DEBUG:shadow.algorithms.heuristic:Ranked 2582/4866
DEBUG:shadow.algorithms.heuristic:Ranked 2796/4866
DEBUG:shadow.algorithms.heuristic:Ranked 2560/4866
DEBUG:shadow.algorithms.heuristic:Ranked 2597/4866
DEBUG:shadow.algorithms.heuristic:Ranked 2799/4866
DEBUG:shadow.algorithms.heuristic:Ranked 2803/4866
DEBUG:shadow.algorithms.heuristic:Ranked 2569/4866
DEBUG:shadow.algorithms.heuristic:Ranked 2582/4866
DEBUG:shadow.algorithms.heuristic:Ranked 2601/4866
DEBUG:shadow.algorithms.heuristic:Ranked 2806/4866
DEBUG:shadow.algorithms.heuristic:Ranked 2597/4866
DEBUG:shadow.algorithms.heuristic:Ranked 2812/4866
Ranking tasks:   4%|███████▋                                                                                                                                                                                                      | 181/486

...

DEBUG:shadow.algorithms.heuristic:Ranked 25094/4866
DEBUG:shadow.algorithms.heuristic:Ranked 25164/4866
DEBUG:shadow.algorithms.heuristic:Ranked 25097/4866
DEBUG:shadow.algorithms.heuristic:Ranked 25163/4866
DEBUG:shadow.algorithms.heuristic:Ranked 25166/4866
DEBUG:shadow.algorithms.heuristic:Ranked 25098/4866
DEBUG:shadow.algorithms.heuristic:Ranked 25164/4866
DEBUG:shadow.algorithms.heuristic:Ranked 25100/4866
Ranking tasks:  98%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████    | 4773/4866 [00:21<00:00, 207.82Tasks/s]DEBUG:shadow.algorithms.heuristic:Ranked 25168/4866
DEBUG:shadow.algorithms.heuristic:Ranked 25101/4866
DEBUG:shadow.algorithms.heuristic:Ranked 25169/4866
DEBUG:shadow.algorithms.heuristic:Ranked 25170/4866
DEBUG:shadow.algorithms.heuristic:Ranked 25171/4866
DEBUG:shadow.algorithms.heuristic:Ranked 25102/4866
DEBUG:s

DEBUG:shadow.algorithms.heuristic:Ranked 25161/4866
DEBUG:shadow.algorithms.heuristic:Ranked 25237/4866
DEBUG:shadow.algorithms.heuristic:Ranked 25163/4866
DEBUG:shadow.algorithms.heuristic:Ranked 25239/4866
DEBUG:shadow.algorithms.heuristic:Ranked 25241/4866
DEBUG:shadow.algorithms.heuristic:Ranked 25236/4866
DEBUG:shadow.algorithms.heuristic:Ranked 25242/4866
DEBUG:shadow.algorithms.heuristic:Ranked 25237/4866
DEBUG:shadow.algorithms.heuristic:Ranked 25243/4866
DEBUG:shadow.algorithms.heuristic:Ranked 25239/4866
DEBUG:shadow.algorithms.heuristic:Ranked 25164/4866
Ranking tasks:  99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉  | 4816/4866 [00:22<00:00, 202.77Tasks/s]DEBUG:shadow.algorithms.heuristic:Ranked 25245/4866
DEBUG:shadow.algorithms.heuristic:Ranked 25166/4866
DEBUG:shadow.algorithms.heuristic:Ranked 25241/4866
DEBUG:s

DEBUG:shadow.algorithms.heuristic:Ranked 25231/4866
DEBUG:shadow.algorithms.heuristic:Ranked 25302/4866
DEBUG:shadow.algorithms.heuristic:Ranked 25297/4866
DEBUG:shadow.algorithms.heuristic:Ranked 25232/4866
DEBUG:shadow.algorithms.heuristic:Ranked 25304/4866
DEBUG:shadow.algorithms.heuristic:Ranked 25305/4866
DEBUG:shadow.algorithms.heuristic:Ranked 25298/4866
DEBUG:shadow.algorithms.heuristic:Ranked 25235/4866
DEBUG:shadow.algorithms.heuristic:Ranked 25299/4866
DEBUG:shadow.algorithms.heuristic:Ranked 25309/4866
DEBUG:shadow.algorithms.heuristic:Ranked 25236/4866
DEBUG:shadow.algorithms.heuristic:Ranked 25310/4866
DEBUG:shadow.algorithms.heuristic:Ranked 25237/4866
Ranking tasks: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 4858/4866 [00:22<00:00, 196.33Tasks/s]DEBUG:shadow.algorithms.heuristic:Ranked 25311/4866
DEBUG:s

Allocating tasks: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4866/4866 [02:44<00:00, 29.57Tasks/s]


Allocating tasks: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4866/4866 [02:45<00:00, 29.32Tasks/s]


In [30]:
HEFTWorkflow = ShadowWorkflow("output/workflows/hpso01_time-18000_channels-256_tel-512_no_data.json")
env = ShadowEnvironment('output/shadow_environment.json')
HEFTWorkflow.add_environment(env)
result = ShadowHEFT(HEFTWorkflow)

ERROR! Session/line number was not unique in database. History logging moved to new session 1457


DEBUG:shadow.models.workflow:Tasks do not have pre calculated runtimes
DEBUG:shadow.algorithms.heuristic:Ranking tasks
Ranking tasks:   0%|                                                                                                                                                                                                                        | 0/43530 [00:00<?, ?Tasks/s]DEBUG:shadow.algorithms.heuristic:Ranked 299/43530
DEBUG:shadow.algorithms.heuristic:Ranked 345/43530
DEBUG:shadow.algorithms.heuristic:Ranked 349/43530
DEBUG:shadow.algorithms.heuristic:Ranked 776/43530
DEBUG:shadow.algorithms.heuristic:Ranked 818/43530
DEBUG:shadow.algorithms.heuristic:Ranked 963/43530
DEBUG:shadow.algorithms.heuristic:Ranked 988/43530
DEBUG:shadow.algorithms.heuristic:Ranked 1081/43530
DEBUG:shadow.algorithms.heuristic:Ranked 1114/43530
DEBUG:shadow.algorithms.heuristic:Ranked 1180/43530
DEBUG:shadow.algorithms.heuristic:Ranked 1220/43530
DEBUG:shadow.algorithms.heuristic:Ranked 1449/43530

DEBUG:shadow.algorithms.heuristic:Ranked 14793/43530
DEBUG:shadow.algorithms.heuristic:Ranked 15072/43530
DEBUG:shadow.algorithms.heuristic:Ranked 15385/43530
DEBUG:shadow.algorithms.heuristic:Ranked 15420/43530
DEBUG:shadow.algorithms.heuristic:Ranked 15470/43530
DEBUG:shadow.algorithms.heuristic:Ranked 15543/43530
DEBUG:shadow.algorithms.heuristic:Ranked 15585/43530
DEBUG:shadow.algorithms.heuristic:Ranked 15589/43530
DEBUG:shadow.algorithms.heuristic:Ranked 15824/43530
Ranking tasks:   0%|▋                                                                                                                                                                                                            | 135/43530 [00:00<05:50, 123.81Tasks/s]DEBUG:shadow.algorithms.heuristic:Ranked 15837/43530
DEBUG:shadow.algorithms.heuristic:Ranked 15928/43530
DEBUG:shadow.algorithms.heuristic:Ranked 16005/43530
DEBUG:shadow.algorithms.heuristic:Ranked 16519/43530
DEBUG:shadow.algorithms.heuristic:Ranked 16542

Ranking tasks:   1%|█                                                                                                                                                                                                             | 231/43530 [00:02<10:52, 66.39Tasks/s]DEBUG:shadow.algorithms.heuristic:Ranked 25281/43530
DEBUG:shadow.algorithms.heuristic:Ranked 25327/43530
DEBUG:shadow.algorithms.heuristic:Ranked 25352/43530
DEBUG:shadow.algorithms.heuristic:Ranked 25540/43530
DEBUG:shadow.algorithms.heuristic:Ranked 25653/43530
DEBUG:shadow.algorithms.heuristic:Ranked 25673/43530
DEBUG:shadow.algorithms.heuristic:Ranked 25828/43530

KeyboardInterrupt



In [16]:
x = _
x.makespan

11244

In [23]:
result.makespan

64986